In [1]:
import pysam
from Bio import bgzf
import io
import gzip

from bai.baiparser import get_bai_bins, get_header_bytes


In [2]:
#block gzip end of file marker
_bgzf_eof = b"\x1f\x8b\x08\x04\x00\x00\x00\x00\x00\xff\x06\x00BC\x02\x00\x1b\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00"

In [3]:
bam = 'data/testdata.bam'
bai = 'data/testdata.bam.bai'


In [4]:
#create header bam dict 

with pysam.AlignmentFile(bam) as b:
    headerobject = b.header

In [5]:
header_end = get_header_bytes(bai)[0]
with open(bam,'rb')as f:
    header_bytes  = f.read(header_end)

In [6]:
chromosome='chr1'
refid= headerobject.get_tid(chromosome)
start = 0
end = 5_000_000

In [7]:
x = get_bai_bins(bai,refid)
start_offset={k:v for n,(k,v) in enumerate(zip(x.keys(),x.values())) if k<=start and list(x.keys())[n+1]>start}
end_offset_k=[k for n,(k,v) in enumerate(zip(x.keys(),x.values())) if k>end ][0]
end_offset={end_offset_k:x[end_offset_k]}
start_startb,start_startoff = bgzf.split_virtual_offset(list(start_offset.values())[0])
end_startb,end_startoff = bgzf.split_virtual_offset(list(end_offset.values())[0])


    
with open(bam,'rb')as f:
    f.seek(start_startb)
    chunk1 = f.read(end_startb-start_startb)
    chunk2 = f.read(20_000)

filehndl = io.BytesIO(chunk1)
values = [x for x  in bgzf.BgzfBlocks(filehndl)]
frst_blck = chunk1[:values[0][1]]
frst_blck_cln = gzip.compress(gzip.decompress(frst_blck)[start_startoff:])
blks_nofirst_nolast = chunk1[values[0][1]:]



filehndl = io.BytesIO(chunk2)

values = []


for n,x in enumerate(bgzf.BgzfBlocks(filehndl)):
    values.append(x)
    break

frst_blck_end = chunk2[:values[0][1]]
frst_blck_end_cln = gzip.compress(gzip.decompress(frst_blck_end)[:end_startoff])
#blks_nofirst_nolast = chunk1[values[0][1]:]

In [8]:
bam_out = f'{chromosome}-{start}-{end}.bam'

In [9]:
with open(bam_out,'wb')as o:
    o.write(header_bytes)
    o.write(frst_blck_cln)
    o.write(blks_nofirst_nolast)
    o.write(frst_blck_end_cln)
    o.write(_bgzf_eof)